In [2]:
import requests
import pandas as pd
import numpy as np

In [3]:
disc_genes="./MLM.DTF.GWAS.Results/summary_genes_discovered.txt"
keywords_list="./mock_keyword_list.txt"

In [4]:
def knetapi(disc_genes):
    with open(disc_genes, "r") as fs:
        next(fs)
        genes = []
        with open(keywords_list, "r") as fl:
            pheno=[]
            for line in fl:
                pheno.append(line.rstrip())
            for line in fs:
                col=line.split("\t")
                genes.append(col[0])
                genelist=(",").join(genes)
                #list of all genes in summary file.            
            
            #obtain knetminer genome API for genetable.
            keyw1="%20OR%20".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
            link="http://knetminer.rothamsted.ac.uk/riceknet/genome?"
            parameters = {"keyword":keyw1, "list":genelist}
            r = requests.get(link, params=parameters)
            
            if not r.ok:
                r.raise_for_status()
                sys.exit()
                
            #extract genetable as tab delimited string from json
            decoded=r.json()[u'geneTable'].split("\t")
            #the last element is a newline or sometimes space. Remove this.
            decoded=(decoded)[:-1]
            
            #tabulate genetable into dataframe by having 9 columns.
            colnum=9
            genetable=np.array(decoded).reshape(len(decoded)//colnum, colnum)
            genetable=pd.DataFrame(genetable[1:,:], columns=genetable[0,:])
        
            
            
knetapi(disc_genes)

ConnectionError: HTTPConnectionPool(host='knetminer.rothamsted.ac.uk', port=80): Max retries exceeded with url: /riceknet/genome?list=Os03g0122600%2COs03g0122600%2COs03g0122600%2COs03g0122600%2COs03g0122600%2COs06g0164900%2COs03g0122600%2COs06g0166100%2COs06g0156700%2COs06g0156600&keyword=%28Germination%29%2520OR%2520%28Coleoptile%2BAND%2Blength%29%2520OR%2520%28Mesocotyl%2BAND%2Blength%29 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000000008929080>: Failed to establish a new connection: [Errno 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))

In [ ]:
def alternate():
    content=pd.read_csv(disc_genes, sep="\t")
    content